In [47]:
%pip install pmdarima

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pmdarima-2.0.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.8 kB)
Using cached pmdarima-2.0.4-cp312-cp312-macosx_11_0_arm64.whl (1.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import necessary libraries 
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
import seaborn as sns
import requests, zipfile
import json 
import fitz
import torch
from collections import defaultdict
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
import os
from io import BytesIO
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display
import pmdarima as pmdarima
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import IPython.display as disp
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

#Get market data from Yahoo Finance
tickers = ["AAPL", "XOM", "TM", "BRK-B", "MCD"]
data = yf.download(tickers, start='2018-01-01', end='2025-03-01')[['Close','Volume']]
data


data_reset = data.reset_index()  
data_reset.to_csv("market_data.csv", index=False)  
market_data = pd.read_csv("market_data.csv") 
market_data['Date'] = pd.to_datetime(market_data['Date'])  
market_data 

market_data = pd.read_csv("market_data.csv")
market_data['Date'] = pd.to_datetime(market_data['Date'], errors="coerce")


market_data = market_data.drop(index=0)

market_data = market_data.rename(columns={
    "Close": "AAPL",
    "Close.1": "BRK-B",
    "Close.2": "MCD",
    "Close.3": "TM",
    "Close.4": "XOM"
})

market_data = market_data[["Date", "AAPL", "BRK-B", "MCD", "TM", "XOM"]]

market_data

for col in ["AAPL", "BRK-B", "MCD", "TM", "XOM"]:
    market_data[col] = pd.to_numeric(market_data[col], errors='coerce')

market_data = market_data.set_index("Date")


# Resample Close prices to monthly (last price of each month)
market_monthly = market_data.resample('M').last()

# Calculate monthly returns
market_returns = market_monthly.pct_change().dropna()

#Extract FOMC minutes from PDF files
folder_path = "/Users/jalenbuffert/Desktop/Capstone/FOMC_Mins"
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

texts = []
for file in files:
    doc = fitz.open(os.path.join(folder_path, file))
    text = ""
    for page in doc:
        text += page.get_text()
        
    filename = file.replace(".pdf", "").lower()
    
    if "fomcminutes" in filename:
        cleaned_filename = filename.replace("fomcminutes", "")
    # Case 2: "fomc20180131meeting" because they are named differently 
    elif "fomc" in filename and "meeting" in filename:
        cleaned_filename = filename.replace("fomc", "").replace("meeting", "")
    else:
        cleaned_filename = filename 
    
    parsed_date = pd.to_datetime(cleaned_filename, format="%Y%m%d", errors="coerce")
    
    texts.append({"date": parsed_date, "text": text})

fomc_df = pd.DataFrame(texts)
fomc_df = fomc_df.dropna(subset=["date"])
fomc_df.sort_values("date").reset_index(drop=True, inplace=True)

fomc_df = fomc_df.sort_values("date").reset_index(drop=True)

## Use FinBERT to extract sentiment from FOMC minutes 
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
model.eval()

def get_finbert_sentiment(text):
    try:
        inputs = tokenizer(text[:512], return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        return {
            "positive": probs[0][0].item(),
            "neutral": probs[0][1].item(),
            "negative": probs[0][2].item()
        }
    except Exception as e:
        return {"positive": None, "neutral": None, "negative": None}


sentiment_scores = []
for text in tqdm(fomc_df["text"], desc="Scoring Sentiment"):
    sentiment_scores.append(get_finbert_sentiment(text))
    
sentiment_df = pd.DataFrame(sentiment_scores)

fomc_df=pd.concat([fomc_df.reset_index(drop=True), sentiment_df.reset_index(drop=True)],axis=1)

fomc_df
fomc_df = fomc_df.set_index("date")

# Resample to Monthly
fomc_monthly = fomc_df.resample("M").ffill()

fomc_monthly.head()
fomc_monthly.describe()

#Extract GDELT sentiment data 
start_date = datetime(2018, 3, 1)
end_date = datetime(2025, 3, 1)

os.makedirs("monthly_sentiment", exist_ok=True)

# Loop through months
current = start_date
while current <= end_date:
    month_start = current.replace(day=1)
    next_month = (month_start + timedelta(days=32)).replace(day=1)
    month_end = next_month - timedelta(days=1)

    filename = f"monthly_sentiment/sentiment_{month_start.strftime('%Y_%m')}.csv"
    if os.path.exists(filename):
        print(f"Already processed: {filename}")
        current = next_month
        continue

    print(f"Processing {month_start.strftime('%B %Y')}...")

    results = []
    for single_date in pd.date_range(month_start, month_end):
        # ⚡ Only 1 timestamp per day (00:00 UTC)
        timestamp = single_date.replace(hour=0).strftime('%Y%m%d%H%M%S')
        url = f"http://data.gdeltproject.org/gdeltv2/{timestamp}.gkg.csv.zip"

        try:
            r = requests.get(url, timeout=10)
            if r.status_code == 200:
                with zipfile.ZipFile(BytesIO(r.content)) as z:
                    file_name = z.namelist()[0]
                    with z.open(file_name) as f:
                        df = pd.read_csv(f, sep='\t', header=None, low_memory=False, encoding='ISO-8859-1')

                        if df.shape[1] >= 21:
                            df = df.iloc[:, :21]
                            df.columns = [
                                "GKGRECORDID", "DATE", "SourceCollectionIdentifier", "SourceCommonName", "DocumentIdentifier",
                                "Themes", "Locations", "Persons", "Organizations", "V2Tone", "Dates", "GCAM", "SharingImage",
                                "RelatedImages", "SocialImageEmbeds", "SocialVideoEmbeds", "Quotations", "AllNames", "Amounts",
                                "TranslationInfo", "Extras"
                            ]
                            
                            for _, row in df.iterrows():
                                tone_str = str(row['V2Tone'])
                                if tone_str and '#' in tone_str:
                                    try:
                                        tone = float(tone_str.split('#')[0])
                                        results.append({
                                            'datetime': single_date,
                                            'tone': tone,
                                            'url': row['DocumentIdentifier']
                                        })
                                    except Exception as e:
                                        print(f"Tone parse error: {e} — Raw tone: {tone_str}")
            else:
                print(f"No file for {timestamp} — Status: {r.status_code}")
        except Exception as e:
            print(f"Error for {timestamp}: {e}")

    sentiment_df = pd.DataFrame(results)
    if not sentiment_df.empty:
        sentiment_df['datetime'] = pd.to_datetime(sentiment_df['datetime'])
        sentiment_df = sentiment_df.dropna(subset=['tone'])

        sentiment_df['year_month'] = sentiment_df['datetime'].dt.to_period('M')
        monthly_sentiment = sentiment_df.groupby('year_month')['tone'].mean().reset_index()
        monthly_sentiment['year_month'] = monthly_sentiment['year_month'].dt.to_timestamp()

        monthly_sentiment.columns = ['date', 'avg_tone']
        monthly_sentiment.to_csv(filename, index=False)
        print(f"Saved {filename}")

    else:
        print(f"No sentiment data for {month_start.strftime('%Y-%m')}")

    current = next_month

#Merge the monthly sentiment data 
all_sentiments = pd.read_csv("all_sentiment_data.csv")

all_sentiments['date'] = pd.to_datetime(all_sentiments['date'])

all_sentiments = all_sentiments.sort_values(by='date').reset_index(drop=True)

all_sentiments

all_sentiments.describe()

all_sentiments = all_sentiments.sort_values(by='date')
fomc_monthly = fomc_monthly.sort_values(by='date')

market_returns = market_returns.reset_index() 
market_returns = market_returns.rename(columns={"Date": "date"}) 
market_returns = market_returns.sort_values(by='date') 

merged_df = all_sentiments.merge(fomc_monthly, on='date', how='outer')
merged_df = merged_df.merge(market_returns, on='date', how='outer')

merged_df = merged_df.sort_values('date').reset_index(drop=True)
merged_df = merged_df.ffill()
merged_df = merged_df.dropna()

merged_df = merged_df.drop(columns=['level_0', 'index', 'index_x', 'index_y'], errors='ignore')
merged_df.head()

#Gather macroeconomic data from FRED
start = '2018-02-01'
end = '2025-03-01'

fred_codes = {
    "Fed_Funds_Rate": "FEDFUNDS",
    "Treasury_10Y_Rate": "DGS10",
    "CPI": "CPIAUCSL",
    "M2_Money_Supply": "M2SL",
    "Unemployment_Rate": "UNRATE",
    "Real_GDP": "GDPC1",
    "Consumer_Sentiment": "UMCSENT"
}

macro_data = {}
for name, code in fred_codes.items():
    try:
        df = web.DataReader(code, "fred", start, end)
        df.columns = [name]
        macro_data[name] = df
        print(f"Downloaded: {name}")
    except Exception as e:
        print(f"Failed to download {name}: {e}")

macro_df = pd.concat(macro_data.values(), axis=1)

macro_df = macro_df.resample('M').mean()

print(macro_df.head())

macro_df.to_csv("macro_data_monthly.csv")
macro_df
macro_df = macro_df.reset_index()  

if 'DATE' in macro_df.columns:
    macro_df = macro_df.rename(columns={'DATE': 'date'})
elif 'index' in macro_df.columns:
    macro_df = macro_df.rename(columns={'index': 'date'})

macro_df['date'] = pd.to_datetime(macro_df['date'])
merged_df['date'] = pd.to_datetime(merged_df['date'])

#Merge the macro data to the rest of the data 
final_df = merged_df.merge(macro_df, on='date', how='left')
final_df = final_df.ffill()

macro_df['date'] = pd.to_datetime(macro_df['date']) + pd.Timedelta(days=1)

macro_df = macro_df.fillna(method='ffill')

final_df = merged_df.merge(macro_df, on='date', how='left')

final_df = final_df.ffill()

if 'date' in final_df.columns:
    final_df['date'] = pd.to_datetime(final_df['date'])
    final_df = final_df.set_index('date')

final_df_clean = final_df.select_dtypes(include=['number'])

final_df_clean.index = final_df_clean.index.to_period('M')

final_df_monthly = final_df_clean.groupby(final_df_clean.index).mean()

final_df_monthly = final_df_monthly.reset_index().rename(columns={'date': 'Month'})

if final_df_monthly.index.name == 'Month':
    final_df_monthly = final_df_monthly.reset_index()

def compute_and_export_stats(df, columns, filename):
    desc = df[columns].describe().T
    desc = desc[['count', 'mean', 'std', 'min', 'max']].round(3)
    desc.to_csv(filename)
    return desc

# All descriptive stats
stock_columns = ['AAPL', 'BRK-B', 'MCD', 'TM', 'XOM']
sentiment_columns = ['avg_tone', 'positive', 'neutral', 'negative']
macro_columns = ['Fed_Funds_Rate', 'Treasury_10Y_Rate', 'CPI', 'M2_Money_Supply', 'Unemployment_Rate', 'Real_GDP', 'Consumer_Sentiment']

desc_stats_stocks = compute_and_export_stats(final_df_monthly, stock_columns, "stock_descriptive_stats.csv")
desc_stats_sentiment = compute_and_export_stats(final_df_monthly, sentiment_columns, "desc_stats_sentiment.csv")
desc_stats_macro = compute_and_export_stats(final_df_monthly, macro_columns, "desc_stats_macro.csv")

numeric_df = final_df_monthly.select_dtypes(include='number')
desc_stats = numeric_df.describe().T
desc_stats['skew'] = numeric_df.skew()
desc_stats['kurtosis'] = numeric_df.kurtosis()
desc_stats = desc_stats.round(3)
desc_stats.to_csv("full_numeric_desc_stats.csv")

#Plot Histograms for each column
for col in stock_columns:
    x = final_df_monthly[col].dropna()
    
    # Calculate stats
    median_val = x.median()
    skew_val = x.skew()
    kurtosis_val = x.kurtosis()
    
    # Create histogram
    fig = px.histogram(
        final_df_monthly, 
        x=col, 
        nbins=50, 
        title=f'Distribution of {col.upper()}',
    )
    
    # Customize layout + bar appearance
    fig.update_traces(
        marker_line_color='black',  # ✅ Thin black outline around bars
        marker_line_width=0.7        # ✅ Very thin (adjust to 1 if you want slightly thicker)
    )
    
    fig.update_layout(
        bargap=0,  # ✅ No gaps between bars
        xaxis_title=col,
        yaxis=dict(
            showticklabels=False,
            title=None
        ),
        title_font_size=20,
        title_x=0.5,
        template="plotly_white"
    )
    
    # Add annotation with stats
    fig.add_annotation(
        xref="paper", yref="paper",
        x=0.95, y=0.95,
        text=f"Median: {median_val:.4f}<br>Skew: {skew_val:.2f}<br>Kurtosis: {kurtosis_val:.2f}",
        showarrow=False,
        align="right",
        font=dict(size=12, color="black"),
        bordercolor="black",
        borderwidth=1,
        borderpad=4,
        bgcolor="white",
        opacity=0.8
    )
    
    fig.show()
    
for col in sentiment_columns:
    raw_data = final_df_monthly[col]

    # If it's a DataFrame with repeated column names, just take the first one
    if isinstance(raw_data, pd.DataFrame):
        x = raw_data.iloc[:, 0]
    else:
        x = raw_data

    x = x.dropna()

    median_val = x.median()
    skew_val = x.skew()
    kurtosis_val = x.kurtosis()

    fig = px.histogram(
        x=x,
        nbins=50,
        title=f'Distribution of {col.upper()}',
    )

    fig.update_traces(
        marker_color='red',
        marker_line_color='black',
        marker_line_width=0.7
    )

    fig.update_layout(
        bargap=0,
        xaxis_title=col,
        yaxis=dict(
            showticklabels=False,
            title=None
        ),
        title_font_size=20,
        title_x=0.5,
        template="plotly_white"
    )

    fig.add_annotation(
        xref="paper", yref="paper",
        x=0.95, y=0.95,
        text=f"Median: {median_val:.4f}<br>Skew: {skew_val:.2f}<br>Kurtosis: {kurtosis_val:.2f}",
        showarrow=False,
        align="right",
        font=dict(size=12, color="black"),
        bordercolor="black",
        borderwidth=1,
        borderpad=4,
        bgcolor="white",
        opacity=0.8
    )

    fig.show()

for col in macro_columns:
    x = final_df_monthly[col].dropna()
    
    median_val = x.median()
    skew_val = x.skew()
    kurtosis_val = x.kurtosis()
    
    fig = px.histogram(
        final_df_monthly, 
        x=col, 
        nbins=50, 
        title=f'Distribution of {col.upper()}',
    )
    
    fig.update_traces(
        marker_color='green',
        marker_line_color='black', 
        marker_line_width=0.7    
    )
    
    fig.update_layout(
        bargap=0, 
        xaxis_title=col,
        yaxis=dict(
            showticklabels=False,
            title=None
        ),
        title_font_size=20,
        title_x=0.5,
        template="plotly_white"
    )
    
    fig.add_annotation(
        xref="paper", yref="paper",
        x=0.95, y=0.95,
        text=f"Median: {median_val:.4f}<br>Skew: {skew_val:.2f}<br>Kurtosis: {kurtosis_val:.2f}",
        showarrow=False,
        align="right",
        font=dict(size=12, color="black"),
        bordercolor="black",
        borderwidth=1,
        borderpad=4,
        bgcolor="white",
        opacity=0.8
    )
    
    fig.show()
    
# Calculate average monthly returns for each stock 
final_df_monthly['Avg_return'] = final_df_monthly[stock_columns].mean(axis=1)

final_df_monthly['Avg_return']

display(final_df_monthly[['Avg_return']])
avg_ret = final_df_monthly['Avg_return'].describe().T 

avg_ret[['count','mean', 'std', 'min', 'max']]

avg_ret.round(3)
avg_ret.to_csv("avg_return_stats.csv")

final_df_monthly = final_df_monthly.loc[:, ~final_df_monthly.columns.duplicated()]
monthly_df = final_df_monthly.copy()
monthly_df['Month'] = monthly_df['Month'].dt.to_timestamp()

from statsmodels.tsa.stattools import adfuller

# Run ADF Test
adf_result = adfuller(final_df_monthly['Avg_return'])

print('ADF Statistic:', adf_result[0])
print('p-value:', adf_result[1])


if adf_result[1] < 0.05:
    print("stationary")
else:
    print("non-stationary")

stepwise_model = auto_arima(
    final_df_monthly['Avg_return'],
    start_p=0, start_q=0,
    max_p=5, max_q=5,
    seasonal=False,    
    trace=True,       
    error_action='ignore',  
    suppress_warnings=True,
    stepwise=True
)

print(stepwise_model.summary())

#Fit ARIMA(3,0,1)
model = ARIMA(final_df_monthly['Avg_return'], order=(3, 0, 1))
results = model.fit()

fitted_values = results.fittedvalues

#Forecast next 6 months
n_periods = 6
forecast = results.get_forecast(steps=n_periods)
forecast_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

#Forecast index (monthly)
forecast_index = pd.date_range(
    start=final_df_monthly['Month'].max().to_timestamp() + pd.offsets.MonthBegin(1),
    periods=n_periods,
    freq='MS'
)


forecast_df = pd.DataFrame({
    'Forecast': forecast_mean.values,
    'Lower CI': conf_int.iloc[:, 0].values,
    'Upper CI': conf_int.iloc[:, 1].values
}, index=forecast_index)

#forecast_df.index = forecast_df.index.to_timestamp()


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=monthly_df['Month'],
    y=final_df_monthly['Avg_return'],
    mode='lines',
    name='Historical',
    line=dict(color='black')
))

fig.add_trace(go.Scatter(
    x=forecast_df.index,
    y=forecast_df['Forecast'],
    mode='lines',
    name='Forecast',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=monthly_df['Month'],
    y=fitted_values,
    mode='lines',
    name='Fitted (ARIMA)',
    line=dict(color='blue')
))

# Confidence Interval
fig.add_trace(go.Scatter(
    x=list(forecast_df.index) + list(forecast_df.index)[::-1],
    y=list(forecast_df['Upper CI']) + list(forecast_df['Lower CI'])[::-1],
    fill='toself',
    fillcolor='rgba(255,182,193,0.3)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=False
))

fig.update_layout(
    title='ARIMA(3,0,1) Forecast of Average Stock Return',
    xaxis_title='Date',
    yaxis_title='Return',
    template='plotly_white'
)

fig.show() 
print(results.summary())
# Coefficient summary table
coef_table_html = results.summary().tables[1].as_html()

coef_df = pd.read_html(coef_table_html, header=0)[0]

coef_df.columns = ['Variable', 'Coefficient', 'Std. Error', 'z', 'P>|z|', '95% CI Lower', '95% CI Upper']

coef_df = coef_df.round(4)


disp.display(coef_df)

# Now select columns again
exog_cols = ['avg_tone', 'positive', 'neutral', 'negative']
exog_vars = final_df_monthly[exog_cols]

# Check again
print(exog_vars.columns)

# Define exogenous variables (sentiment inputs)
exog_vars = final_df_monthly[['avg_tone', 'positive', 'neutral', 'negative']]

# Define target variable
target = final_df_monthly['Avg_return']

# Fit SARIMAX model with ARIMA(3,0,1) + exogenous regressors
sarimax_model = SARIMAX(
    target,
    exog=exog_vars,
    order=(3, 0, 1),
    enforce_stationarity=False,
    enforce_invertibility=False
)

sarimax_results = sarimax_model.fit(disp=False)

# Display summary
print(sarimax_results.summary())
coef_table_html = sarimax_results.summary().tables[1].as_html()

coef_df = pd.read_html(coef_table_html, header=0)[0]

coef_df.columns = ['Variable', 'Coefficient', 'Std. Error', 'z', 'P>|z|', '95% CI Lower', '95% CI Upper']

coef_df = coef_df.round(4)

disp.display(coef_df)
# 1. Get SARIMAX forecast
n_periods = 6
last_exog = final_df_monthly[['avg_tone', 'positive', 'neutral', 'negative']].iloc[[-1]].values
future_exog = pd.DataFrame(last_exog.repeat(n_periods, axis=0),
                           columns=['avg_tone', 'positive', 'neutral', 'negative'])

# 2. Forecast future dates
forecast_index = pd.date_range(
    start=final_df_monthly['Month'].max().to_timestamp() + pd.offsets.MonthBegin(1),
    periods=n_periods,
    freq='MS'
)

# 3. Get forecast and confidence intervals
sarimax_forecast = sarimax_results.get_forecast(steps=n_periods, exog=future_exog)
forecast_mean = sarimax_forecast.predicted_mean
conf_int = sarimax_forecast.conf_int(alpha=0.05)  # 95% CI
conf_int.columns = ['Lower CI', 'Upper CI']
conf_int.index = forecast_index  # Ensure index alignment

# 4. Forecast DataFrame
forecast_df = pd.DataFrame({
    'Forecast': forecast_mean.values,
    'Lower CI': conf_int['Lower CI'].values,
    'Upper CI': conf_int['Upper CI'].values
}, index=forecast_index)

# 5. Plot using Plotly
fig = go.Figure()

# Observed
fig.add_trace(go.Scatter(
    x=monthly_df['Month'],
    y=final_df_monthly['Avg_return'],
    mode='lines',
    name='Observed',
    line=dict(color='black')
))

# Fitted
fig.add_trace(go.Scatter(
    x=monthly_df['Month'],
    y=sarimax_results.fittedvalues,
    mode='lines',
    name='Fitted (SARIMAX)',
    line=dict(color='blue')
))

# Forecast
fig.add_trace(go.Scatter(
    x=forecast_df.index,
    y=forecast_df['Forecast'],
    mode='lines',
    name='Forecast',
    line=dict(color='red')
))

# Confidence Interval
fig.add_trace(go.Scatter(
    x=list(forecast_df.index) + list(forecast_df.index)[::-1],
    y=list(forecast_df['Upper CI']) + list(forecast_df['Lower CI'])[::-1],
    fill='toself',
    fillcolor='rgba(255,182,193,0.3)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=False
))

fig.update_layout(
    title='SARIMAX Forecast with Sentiment Features',
    xaxis_title='Date',
    yaxis_title='Average Return',
    template='plotly_white',
    width=1000,
    height=500,
    legend=dict(x=0.01, y=0.99)
)

fig.show()
#conda install -c conda-forge py-xgboost


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  5 of 5 completed
/var/folders/s1/n0q5fh9s5m77hjt_c78j17w00000gn/T/ipykernel_56904/1600136521.py:79: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  market_monthly = market_data.resample('M').last()
Scoring Sentiment: 100%|██████████| 58/58 [00:03<00:00, 15.29it/s]
/var/folders/s1/n0q5fh9s5m77hjt_c78j17w00000gn/T/ipykernel_56904/1600136521.py:147: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  fomc_monthly = fomc_df.resample("M").ffill()


Already processed: monthly_sentiment/sentiment_2018_03.csv
Already processed: monthly_sentiment/sentiment_2018_04.csv
Already processed: monthly_sentiment/sentiment_2018_05.csv
Already processed: monthly_sentiment/sentiment_2018_06.csv
Already processed: monthly_sentiment/sentiment_2018_07.csv
Already processed: monthly_sentiment/sentiment_2018_08.csv
Already processed: monthly_sentiment/sentiment_2018_09.csv
Already processed: monthly_sentiment/sentiment_2018_10.csv
Already processed: monthly_sentiment/sentiment_2018_11.csv
Already processed: monthly_sentiment/sentiment_2018_12.csv
Already processed: monthly_sentiment/sentiment_2019_01.csv
Already processed: monthly_sentiment/sentiment_2019_02.csv
Already processed: monthly_sentiment/sentiment_2019_03.csv
Already processed: monthly_sentiment/sentiment_2019_04.csv
Already processed: monthly_sentiment/sentiment_2019_05.csv
Already processed: monthly_sentiment/sentiment_2019_06.csv
Already processed: monthly_sentiment/sentiment_2019_07.c

/var/folders/s1/n0q5fh9s5m77hjt_c78j17w00000gn/T/ipykernel_56904/1600136521.py:285: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  macro_df = macro_df.resample('M').mean()
/var/folders/s1/n0q5fh9s5m77hjt_c78j17w00000gn/T/ipykernel_56904/1600136521.py:307: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  macro_df = macro_df.fillna(method='ffill')


,Avg_return
0,-0.031882
1,-0.005979
2,0.018554
3,0.008737
4,0.007778
...,...
80,0.001456
81,0.017209
82,-0.003254
83,0.013843


...

/var/folders/s1/n0q5fh9s5m77hjt_c78j17w00000gn/T/ipykernel_56904/1600136521.py:627: FutureWarning:

Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



,Variable,Coefficient,Std. Error,z,P>|z|,95% CI Lower,95% CI Upper
0,const,0.0137,0.0050,3.014,0.003,0.005,0.023
1,ar.L1,-0.2090,0.0940,-2.220,0.026,-0.394,-0.024
2,ar.L2,-0.2263,0.1400,-1.613,0.107,-0.501,0.049
3,ar.L3,0.0856,0.1670,0.514,0.607,-0.241,0.412
4,ma.L1,0.9729,0.0930,10.419,0.000,0.790,1.156
5,sigma2,0.0005,0.0001,6.621,0.000,0.000,0.001


Index(['avg_tone', 'positive', 'neutral', 'negative'], dtype='object')
                               SARIMAX Results                                
Dep. Variable:             Avg_return   No. Observations:                   85
Model:               SARIMAX(3, 0, 1)   Log Likelihood                 190.725
Date:                Fri, 06 Jun 2025   AIC                           -363.449
Time:                        12:36:25   BIC                           -341.789
Sample:                             0   HQIC                          -354.753
                                 - 85                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
avg_tone      -0.0005      0.075     -0.007      0.995      -0.147       0.146
positive       0.0166      0.165      0.100      0.920      

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/var/folders/s1/n0q5fh9s5m77hjt_c78j17w00000gn/T/ipykernel_56904/1600136521.py:664: FutureWarning:

Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



,Variable,Coefficient,Std. Error,z,P>|z|,95% CI Lower,95% CI Upper
0,avg_tone,-0.0005,0.075,-0.007,0.995,-0.147,0.146
1,positive,0.0166,0.165,0.100,0.920,-0.308,0.341
2,neutral,-1041.9410,283.443,-3.676,0.000,-1597.479,-486.403
3,negative,528.6253,632.445,0.836,0.403,-710.945,1768.195
4,ar.L1,-0.2398,0.102,-2.350,0.019,-0.440,-0.040
5,ar.L2,-0.2418,0.151,-1.606,0.108,-0.537,0.053
6,ar.L3,0.0635,0.174,0.365,0.715,-0.278,0.405
7,ma.L1,0.9877,0.204,4.846,0.000,0.588,1.387
8,sigma2,0.0005,0.000,4.816,0.000,0.000,0.001


In [21]:
monthly_df

,Month,avg_tone,positive,neutral,negative,AAPL,BRK-B,MCD,TM,XOM,Fed_Funds_Rate,Treasury_10Y_Rate,CPI,M2_Money_Supply,Unemployment_Rate,Real_GDP,Consumer_Sentiment,Avg_return
0,2018-03-01,2.234360,0.999997,1.531594e-06,1.395896e-06,0.005067,-0.035375,-0.040545,-0.019326,-0.069232,1.42,2.858947,249.529,13919.3,4.1,NaN,99.7,-0.031882
1,2018-04-01,2.193257,0.999996,2.144961e-06,1.945623e-06,-0.036535,-0.033042,0.031052,-0.004950,0.013583,1.51,2.842381,249.577,13976.0,4.0,NaN,101.4,-0.005979
2,2018-05-01,2.217867,0.999996,1.686396e-06,1.864139e-06,0.060052,-0.020090,0.013176,-0.009151,0.048782,1.69,2.869048,250.227,13993.9,4.0,20150.476,98.8,0.018554
3,2018-06-01,2.190757,0.999997,1.392207e-06,1.636908e-06,0.062853,-0.018418,-0.029451,-0.008208,0.036910,1.70,2.977727,250.792,14049.6,3.8,20150.476,98.0,0.008737
4,2018-07-01,2.215917,0.999997,1.556583e-06,1.491162e-06,0.009282,0.017317,-0.004552,0.015043,0.001797,1.82,2.912381,251.018,14109.9,4.0,20150.476,98.2,0.007778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,2024-11-01,2.256368,0.999998,7.531309e-08,1.823286e-06,0.010639,0.025447,-0.013685,-0.022447,0.007326,4.83,4.095455,315.564,21323.3,4.1,23542.349,70.5,0.001456
81,2024-12-01,2.277332,0.999996,5.201588e-07,3.150619e-06,0.053431,0.004808,-0.000716,0.063362,-0.034838,4.64,4.355789,316.449,21424.0,4.2,23542.349,71.8,0.017209
82,2025-01-01,2.261643,0.999995,2.729380e-06,2.266651e-06,-0.001214,-0.013809,-0.009444,0.055676,-0.047480,4.48,4.391429,317.603,21441.8,4.1,23542.349,74.0,-0.003254
83,2025-02-01,2.241172,0.999994,4.525562e-06,1.066282e-06,-0.015855,0.065155,0.031945,-0.034310,0.022283,4.33,4.629048,319.086,21520.0,4.0,23528.047,71.7,0.013843


In [22]:
# New pipeline without PCA
pipeline_no_pca = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=1))
])

# Define features and target
X = monthly_df.drop(columns=['Month', 'Avg_return','AAPL', 'BRK-B', 'MCD', 'TM', 'XOM'])
y = monthly_df['Avg_return']

# Fit model
pipeline_no_pca.fit(X, y)

# Predictions
y_pred = pipeline_no_pca.predict(X)

# MSE
xgb_md = mean_squared_error(y, y_pred)
print("MSE:", xgb_md)

# Plot
fig = go.Figure()

# Actual values
fig.add_trace(go.Scatter(
    x=monthly_df['Month'],
    y=y,
    mode='lines',
    name='Actual',
    line=dict(color='black')
))

# Predicted values
fig.add_trace(go.Scatter(
    x=monthly_df['Month'],
    y=y_pred,
    mode='lines',
    name='Predicted (XGBoost)',
    line=dict(color='green')
))

# Layout
fig.update_layout(
    title='XGBoost Predictions Without PCA',
    xaxis_title='Date',
    yaxis_title='Avg Return',
    template='plotly_white',
    width=900,
    height=500,
    legend=dict(x=0.01, y=0.99),
    margin=dict(l=40, r=40, t=40, b=40)
)

fig.show()

MSE: 7.512113017647077e-07


In [23]:
# Create figure
fig = go.Figure()

# Scatter of actual vs predicted
fig.add_trace(go.Scatter(
    x=y,
    y=y_pred,
    mode='markers',
    name='Predicted vs Actual',
    marker=dict(opacity=0.7, size = 11)
))

# Perfect fit line
min_val = min(min(y), min(y_pred))
max_val = max(max(y), max(y_pred))
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    name='Perfect Fit',
    line=dict(color='red', dash='dash')
))

# Layout
fig.update_layout(
    title='Actual vs Predicted Returns',
    xaxis_title='Actual',
    yaxis_title='Predicted',
    template='plotly_white',
    width=600,
    height=600,
    showlegend=False
)

fig.show()

In [24]:
import plotly.express as px
import pandas as pd

# Extract feature importances
model = pipeline_no_pca.named_steps['xgb']
feature_importance = model.feature_importances_

# Get raw feature names
feature_names = X.columns.tolist()
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False).head(10)

# Plot
px.bar(importance_df, x='Importance', y='Feature', orientation='h',
       title='Top 10 Features (XGBoost)',
       template='plotly_white')


In [26]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def evaluate_model(true, pred, label):
    metrics = {
        "Model": label,
        "RMSE": np.sqrt(mean_squared_error(true, pred)),
        "MAE": mean_absolute_error(true, pred),
        "R2": r2_score(true, pred),
        "Directional_Accuracy": np.mean(np.sign(true) == np.sign(pred)) * 100
    }
    return metrics

# Evaluate each model
arima_metrics = evaluate_model(y, results.fittedvalues, "ARIMA")
sarimax_metrics = evaluate_model(y, sarimax_results.fittedvalues, "SARIMAX")
xgb_metrics = evaluate_model(y, y_pred, "XGBoost")

# Combine into DataFrame for comparison
import pandas as pd
comparisons_df = pd.DataFrame([arima_metrics, sarimax_metrics, xgb_metrics])
print(comparisons_df)

     Model      RMSE       MAE        R2  Directional_Accuracy
0    ARIMA  0.023667  0.017609  0.414465             72.941176
1  SARIMAX  0.023922  0.017917  0.401749             75.294118
2  XGBoost  0.000867  0.000624  0.999215            100.000000


In [27]:
disp.display(comparisons_df)

,Model,RMSE,MAE,R2,Directional_Accuracy
0,ARIMA,0.023667,0.017609,0.414465,72.941176
1,SARIMAX,0.023922,0.017917,0.401749,75.294118
2,XGBoost,0.000867,0.000624,0.999215,100.000000


In [29]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# --- CONFIGURABLE PARAMETERS ---
transaction_cost = 0.001         # 0.1% per trade
slippage_std = 0.0005            # Simulate slippage (50 bps)
use_confidence_weighting = True # Use predicted confidence as position size
use_shorting = False             # Allow shorting?

# --- SIGNAL GENERATION ---
if use_confidence_weighting:
    signals = np.tanh(y_pred * 10)
else:
    signals = np.where(y_pred > 0, 1, -1 if use_shorting else 0)

# --- APPLY SLIPPAGE & COSTS ---
slippage = np.random.normal(0, slippage_std, size=len(y))
gross_returns = signals * y
net_returns = gross_returns - np.abs(signals) * (transaction_cost + slippage)

# --- CAPITAL GROWTH ---
capital = [1]
for r in net_returns:
    capital.append(capital[-1] * (1 + r))
capital = np.array(capital[1:])
cumulative_returns = capital - 1

# --- METRICS ---
sharpe_ratio = np.mean(net_returns) / np.std(net_returns) * np.sqrt(12)
drawdown = np.maximum.accumulate(capital) - capital
max_drawdown = np.max(drawdown)

print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Max Drawdown: {max_drawdown:.2%}")

# --- PLOTLY CHART ---
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=monthly_df['Month'],
    y=cumulative_returns,
    mode='lines',
    name='Cumulative Return',
    line=dict(color='green')
))

fig.update_layout(
    title='Sentiment-Based Strategy: Cumulative Returns',
    xaxis_title='Date',
    yaxis_title='Cumulative Return',
    template='plotly_white',
    width=950,
    height=500,
    showlegend=True,
    annotations=[
        dict(
            x=monthly_df.iloc[-1],
            y=cumulative_returns[-1],
            text=f"Sharpe: {sharpe_ratio:.2f}<br>Max Drawdown: {max_drawdown:.2%}",
            showarrow=True,
            arrowhead=1,
            ax=-40,
            ay=-40,
            bgcolor="white",
            bordercolor="black",
            borderwidth=1
        )
    ]
)

fig.show()

Sharpe Ratio: 2.26
Max Drawdown: 0.00%


In [32]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

# 1. Download SPY
start_date = monthly_df['Month'].min()
end_date = monthly_df['Month'].max() + pd.offsets.MonthEnd(1)
spy = yf.download('SPY', start=start_date, end=end_date)

# 2. Resample to monthly, get close price, compute returns
spy_monthly = spy['Close'].resample('M').last()
spy_returns = spy_monthly.pct_change().dropna()
spy_returns.name = 'SPY_Return'  # ✅ Properly name the series

# 3. Convert index to match strategy_df
spy_returns.index = spy_returns.index.to_period('M').to_timestamp()

# 4. Create strategy DataFrame
strategy_df = pd.DataFrame({
    'Month': monthly_df['Month'],
    'True_Return': y.values,
    'Predicted_Return': y_pred
}).set_index('Month')

# 5. Align indices and join SPY returns
spy_returns = spy_returns.reindex(strategy_df.index)  # Align indices
strategy_df = strategy_df.join(spy_returns, how='left')

# 6. Signals and strategy logic
strategy_df['Signal'] = (strategy_df['Predicted_Return'] > 0).astype(int)
strategy_df['Strategy_Return'] = strategy_df['Signal'] * strategy_df['True_Return']

# 7. Cumulative returns
strategy_df['Cumulative_Strategy'] = (1 + strategy_df['Strategy_Return']).cumprod()
strategy_df['Cumulative_SPY'] = (1 + strategy_df['SPY']).cumprod()

# 8. Plot with Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=strategy_df.index,
    y=strategy_df['Cumulative_Strategy'],
    mode='lines',
    name='Sentiment-Based Strategy',
    line=dict(color='green')
))

fig.add_trace(go.Scatter(
    x=strategy_df.index,
    y=strategy_df['Cumulative_SPY'],
    mode='lines',
    name='SPY Benchmark',
    line=dict(color='gray')
))

fig.update_layout(
    title='Cumulative Returns: Sentiment Strategy vs SPY',
    yaxis=dict(
        title='Cumulative Return (%)',
        tickformat=".0%"),
    xaxis_title='Date',
    yaxis_title='Cumulative Return',
    template='plotly_white',
    width=950,
    height=500,
    legend=dict(x=0.02, y=0.98)
)

fig.show()

[*********************100%***********************]  1 of 1 completed
/var/folders/s1/n0q5fh9s5m77hjt_c78j17w00000gn/T/ipykernel_56904/2000597005.py:11: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [33]:
def sharpe_ratio(returns, risk_free_rate=0):
    excess_returns = returns - risk_free_rate
    return excess_returns.mean() / excess_returns.std()

def max_drawdown(cumulative_returns):
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    return drawdown.min()

def hit_rate(returns):
    return (returns > 0).mean() * 100

def profit_factor(returns):
    gains = returns[returns > 0].sum()
    losses = -returns[returns < 0].sum()
    return gains / losses if losses != 0 else np.nan

In [34]:
def evaluate_trading_metrics(strategy_returns):
    cumulative = (1 + strategy_returns).cumprod()
    return {
        "Sharpe Ratio": sharpe_ratio(strategy_returns),
        "Max Drawdown": max_drawdown(cumulative),
        "Hit Rate (%)": hit_rate(strategy_returns),
        "Profit Factor": profit_factor(strategy_returns)
    }

# Example
metrics = evaluate_trading_metrics(strategy_df['Strategy_Return'])
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")
    
spy_metrics = evaluate_trading_metrics(strategy_df['SPY'].dropna())

print("\nSPY Benchmark Metrics:")
for k, v in spy_metrics.items():
    print(f"{k}: {v:.4f}")

Sharpe Ratio: 0.9839
Max Drawdown: 0.0000
Hit Rate (%): 72.9412
Profit Factor: nan

SPY Benchmark Metrics:
Sharpe Ratio: 0.2323
Max Drawdown: -0.2393
Hit Rate (%): 66.6667
Profit Factor: 1.7603


In [35]:

# Build DataFrame from metrics
comparison_df = pd.DataFrame({
    'Sentiment Strategy': metrics,
    'SPY Benchmark': spy_metrics
})

# Optional: round for display
comparison_df = comparison_df.round(4)

# Show table
print(comparison_df)

               Sentiment Strategy  SPY Benchmark
Sharpe Ratio               0.9839         0.2323
Max Drawdown               0.0000        -0.2393
Hit Rate (%)              72.9412        66.6667
Profit Factor                 NaN         1.7603


In [36]:
disp.display(comparison_df)

,Sentiment Strategy,SPY Benchmark
Sharpe Ratio,0.9839,0.2323
Max Drawdown,0.0000,-0.2393
Hit Rate (%),72.9412,66.6667
Profit Factor,NaN,1.7603
